# Entity Resolution Prototype (Probabilistic) 

We're going to slowly walk through this thing and try to replicate [these guys](http://crpit.com/confpapers/CRPITV159Moir.pdf). The approach will be to start simple
and then walk our way into more complex algorithms. If all goes well, this notebook
will demonstrate an implementation that can then be expressed in Java. The prototyping steps
are as follows:

1. Getting Data with (including labeled training and CV data)
1. Cleaning Data
1. Metrics or Algorithms for Comparing String Values
1. Train SVM
1. Tune SVM with genetic algorithm
1. Figure out bad values with Q-learning algorithm
1. Construct Bayesian network of generic feature resolvers
1. Use genetic algorithm to find optimal Bayes Network

Note that steps 3-6 are required to construct a 'generic feature resolver'. Steps
7-8 are required to construct a bayesian network of those feature resolvers. There
is a second notebook that will provide a deterministic approach rather than the 
probabilistic one you see here.

## Step 1: Getting Data with (including labeled training and CV data)

In [1]:
# Data was taken from the Kaggle Titanic competition (https://www.kaggle.com/c/titanic)

import pandas as pd

pd.read_csv("../data/train.csv")[0:5]


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


We know that there aren't multiple instances of the same person here. However, there are instances of the
same family. So our goal is to resolve entities in this dataset to the **family** level; not the individual. We 
would like our unit of analysis to be the family.

In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

print("Training set rows: {}".format(len(train)))
print("Test set rows: {}".format(len(test)))

# Note that there isn't a cross validation set available unless we make one.

Training set rows: 891
Test set rows: 418


Unfortunately, this data doesn't include labeled training data for the purpose of entity
resolution. It's not too much so I'll go in and do it by hand. Remember, our goal is to use
entity resolution to aggregate from the *individual* to the *family* unit of analysis.

In [3]:
import re

def labelCheater(name):
    """ Life's short, we don't have to copy/paste; just verify """
    pattern = re.compile("(.*)(?=,)")
    
    try:
        return pattern.match(name).group(1)
    except:
        return "NotGood"
    

In [4]:
train["familyLabel"] = train["Name"].apply((lambda name: labelCheater(name)))
test["familyLabel"] = test["Name"].apply((lambda name: labelCheater(name)))

In [5]:
# Now instead of copy/pasting; I'll just manually inspect the labels 

train[["Name","familyLabel"]][165:180]

,Name,familyLabel
165,"Goldsmith, Master. Frank John William ""Frankie""",Goldsmith
166,"Chibnall, Mrs. (Edith Martha Bowerman)",Chibnall
167,"Skoog, Mrs. William (Anna Bernhardina Karlsson)",Skoog
168,"Baumann, Mr. John D",Baumann
169,"Ling, Mr. Lee",Ling
170,"Van der hoef, Mr. Wyckoff",Van der hoef
171,"Rice, Master. Arthur",Rice
172,"Johnson, Miss. Eleanor Ileen",Johnson
173,"Sivola, Mr. Antti Wilhelm",Sivola
174,"Smith, Mr. James Clinch",Smith


In [6]:
# we'll just need to do the same thing to get a score for the test set
# at the end of the experiment.

test[["Name","familyLabel"]][0:5]

,Name,familyLabel
0,"Kelly, Mr. James",Kelly
1,"Wilkes, Mrs. James (Ellen Needs)",Wilkes
2,"Myles, Mr. Thomas Francis",Myles
3,"Wirz, Mr. Albert",Wirz
4,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",Hirvonen


There's some strangeness with parentheses that I may have to 
adjust at some point but for now we'll leave it in because
we want our noise to be **loud**.

##Step 2: Cleaning Data

In [7]:
import string
from nltk.corpus import stopwords



def cleanNames(name):
    """ Clean names by removing punctuation and stop words, also uppercase everything"""
    
    stop = stopwords.words('english')
    clean_string = ""
    # may need to remove things in parentheses at some point
    
    for word in name.split("\s"):
        if word not in stop:
            word = word.translate(None, string.punctuation)
            clean_string += word + " "
            
    return clean_string[0:-1].upper()
            

In [8]:
train["Name"] = train["Name"].apply((lambda x : cleanNames(x)))
train["familyLabel"] = train["familyLabel"].apply((lambda x : cleanNames(x)))
test["Name"] = test["Name"].apply((lambda x : cleanNames(x)))
test["familyLabel"] = test["familyLabel"].apply((lambda x : cleanNames(x)))

# training data
train[0:5]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,familyLabel
0,1,0,3,BRAUND MR OWEN HARRIS,male,22,1,0,A/5 21171,7.2500,NaN,S,BRAUND
1,2,1,1,CUMINGS MRS JOHN BRADLEY FLORENCE BRIGGS THAYER,female,38,1,0,PC 17599,71.2833,C85,C,CUMINGS
2,3,1,3,HEIKKINEN MISS LAINA,female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,HEIKKINEN
3,4,1,1,FUTRELLE MRS JACQUES HEATH LILY MAY PEEL,female,35,1,0,113803,53.1000,C123,S,FUTRELLE
4,5,0,3,ALLEN MR WILLIAM HENRY,male,35,0,0,373450,8.0500,NaN,S,ALLEN


In [9]:
# test data
test[0:5]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,familyLabel
0,892,3,KELLY MR JAMES,male,34.5,0,0,330911,7.8292,NaN,Q,KELLY
1,893,3,WILKES MRS JAMES ELLEN NEEDS,female,47.0,1,0,363272,7.0000,NaN,S,WILKES
2,894,2,MYLES MR THOMAS FRANCIS,male,62.0,0,0,240276,9.6875,NaN,Q,MYLES
3,895,3,WIRZ MR ALBERT,male,27.0,0,0,315154,8.6625,NaN,S,WIRZ
4,896,3,HIRVONEN MRS ALEXANDER HELGA E LINDQVIST,female,22.0,1,1,3101298,12.2875,NaN,S,HIRVONEN


In [10]:
from pandasql import *

pysqldf = lambda q: sqldf(q, globals())

q1 = """
    SELECT familyLabel, 
            COUNT(familyLabel) as freq
    FROM train
    GROUP BY
        familyLabel
    ORDER BY
        COUNT(familyLabel) DESC;
"""
q2 = """
    SELECT familyLabel, 
            COUNT(familyLabel) as freq
    FROM test
    GROUP BY
        familyLabel
    ORDER BY
        COUNT(familyLabel) DESC;
"""

df = pysqldf(q1)

#test["familyLabel"].groupby(test["familyLabel"]).count()

In [11]:
df[0:5]

,familyLabel,freq
0,ANDERSSON,9
1,SAGE,7
2,CARTER,6
3,GOODWIN,6
4,JOHNSON,6


In [12]:
print("Training: Number of families: {}".format(len(df)))
print("Training: Number of family memebers: {}".format(len(train["familyLabel"])))

Training: Number of families: 667
Training: Number of family memebers: 891


In [13]:
df = pysqldf(q2)
df[0:5]

,familyLabel,freq
0,ASPLUND,4
1,DAVIES,4
2,SAGE,4
3,THOMAS,4
4,WARE,4


In [14]:
print("Test: Number of families: {}".format(len(df)))
print("Test: Number of family memebers: {}".format(len(test["familyLabel"])))

Test: Number of families: 352
Test: Number of family memebers: 418


##Step 3: Metrics or Algorithms for Comparing String Values

The paper we cited specifies the following in regards to comparing string values:

    "The GER framework has a library of different metrics, or algorithms, for comparing
    string values, including: Dice's coefficient; Soundex; Metaphone; Caverphone;
    regular expression comparison; exact match; Levenshtein; Nysiis; and difflibratio, 
    which is the sequence similarity ratio calculated by the SequenceMatcher class in 
    the difflib Python library." (p. 50)
    
These comparison metrics all accomplish slightly different things. Given that we expect,
strings that are mostly different except for the last name (which may be mispelled) the
Levenshtein edit-distance algorithm makes the most sense. Let's also try difflibratio as well. We are
looking for algorithms that focus on string similarity rather than how those strings 
sound when pronounced.

In [15]:
from nltk.metrics.distance import edit_distance
from difflib import SequenceMatcher

# let's show a couple examples
sm = SequenceMatcher(train["Name"][0], train["familyLabel"][0])

print("Edit distance: {}".format(edit_distance(train["Name"][0],train["familyLabel"][0])))
print("DiffLibRatio: {}".format(sm.ratio()))


Edit distance: 15
DiffLibRatio: 0.0


##Step 4: Train SVM

The paper specifies two different configurations for the GER framework:

1. Used only the basic attributes of the movie, such as the title or synopsis
1. Used the basic attributes of the movie and the asssociation between a movie and its actors to determine if the two movies matched. 

This framework appears to be missing a couple details; namely:

1. Is this framework run for each potential pair?

I'll start by modifying the paper slightly just to get something working then revise later.

In [16]:
# token frequency, inverse document frequency (TF IDF) with svm using whitespace tokenizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

from nltk.tokenize import WhitespaceTokenizer

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt


In [17]:
# configure tokenizer

def tokenize(string):
    """ Returns list after whitespace tokenization"""
    return WhitespaceTokenizer().tokenize(string)


In [18]:
# define a pipeline combining a text feature extractor with a simple
# classifier
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                         alpha=1e-3, n_iter=5, random_state=42))
])


In [19]:
_ = text_clf.fit(train["Name"], train["familyLabel"])

In [20]:
test["prediction"] = text_clf.predict(test["Name"])

In [21]:


def happyScatter(df, colname):
    """ Turn strings into a happy scatterplot """
    nums  = {}
    grouping = 1
    
    for name in df[colname]:
        if name not in nums:
            nums[name] = 1
        else:
            nums[name] += 1
            
    newcol = []
    
    for group in nums.keys():
        gr = grouping
        
        while nums[group] > 0:
            newcol.append(gr)
            nums[group] -= 1
            
        grouping += 1
            
    return newcol
    
    
    
    

In [22]:
test['scatterPredict'] = happyScatter(test, 'prediction')
test['scatterFamilyLabel'] = happyScatter(test, 'familyLabel')
test['scatterName'] = happyScatter(test, 'Name')

In [23]:
test[["familyLabel", "prediction"]][0:5]

,familyLabel,prediction
0,KELLY,KELLY
1,WILKES,HOCKING
2,MYLES,STEAD
3,WIRZ,DICK
4,HIRVONEN,HIRVONEN


In [25]:
# plotting the svm classifiers
#plt.figure(figsize=(8, 8))

#plt.scatter()
#plt.show()

In [31]:
test["predictCorrect"].applymap((lambda x: 1 if test["prediction"] == test["familyLabel"] else 0))

KeyError: 'predictCorrect'

In [ ]:
def evaluation(name, prediction, familyLabel):
    """ Evaluate as proportion correctly labeled """
    score = float(name[prediction == familyLabel].count())/ name.count()
    return round((score * 100),2)

In [ ]:
evaluation(test["Name"], test["prediction"], test["familyLabel"])

##Step 5: Tune SVM with genetic algorithm

##Step 6: Figure out bad values with Q-learning algorithm

##Step 7: Construct Bayesian network of generic feature resolvers

##Step 8: Use genetic algorithm to find optimal Bayes Network